# Day 2 Machine Learning in Python - exercises with answers

## Exercise 1

#### Question 1 
##### Import the required packages to perform K-nearest neighbours.

#### Answer:


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# New today - we will introduce it when we use it.
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import metrics


#### Question 2 
##### Set `main_dir` to the location of your `skillsoft-intro-to-machine-learning-in-python` folder.
##### Make `data_dir` from the `main_dir` and concatenate remainder of the path to data directory.

#### Answer:

In [2]:
from pathlib import Path
# Set `home_dir` to the root directory of your computer.
home_dir = Path.home()
# Set `main_dir` to the location of your `skillsoft-intro-to-machine-learning-in-python` folder.
main_dir = home_dir / "Desktop" / "skillsoft-intro-to-machine-learning-in-python"
# Make `data_dir` from the `main_dir` and remainder of the path to data directory.
data_dir = main_dir / "data"
# Create a plot directory to save our plots
plot_dir = main_dir / "plots"

#### Question 3 
##### Set the working directory to `data_dir`.
##### Check if the working directory is updated to `data_dir'.

#### Answer:

In [3]:
# Change the working directory.
os.chdir(data_dir)

# Check the working directory.
print(os.getcwd())

C:\Users\aashe\Desktop\skillsoft-intro-to-machine-learning-in-python\data


#### Question 4 
##### Load the `chicago_census.csv` dataset. 
##### Save it as `chicago_census` and print the few rows

#### Answer:

In [4]:
chicago_census = pd.read_csv("chicago_census.csv")

# Look at the first five rows using .head().
chicago_census.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


#### Question 5 
###### Rename the column names as the following (and double check they are renamed by looking at `.columns`):
###### c("community_number", "community_area", "percent_house_crowded", "percent_house_below_poverty",
###### "percent_16_unemployed", "percent_25_without_diploma", "percent_dependent", "per_capita_income",
###### "hardship_index")

###### Hint: Use the code below to rename the columns.

In [5]:
chicago_census.rename(columns={'Community Area Number': 'community_number',
                               'COMMUNITY AREA NAME': 'community_area' , 
                               'PERCENT OF HOUSING CROWDED': 'percent_house_crowded', 
                               'PERCENT HOUSEHOLDS BELOW POVERTY': 'percent_house_below_poverty', 
                               'PERCENT AGED 16+ UNEMPLOYED': 'percent_16_unemployed' , 
                               'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA': 'percent_25_without_diploma', 
                               'PERCENT AGED UNDER 18 OR OVER 64': 'percent_dependent', 
                               'PER CAPITA INCOME ' : 'per_capita_income',
                               'HARDSHIP INDEX': 'hardship_index'}, inplace=True)

#### Answer:

In [6]:
chicago_census.rename(columns={'Community Area Number': 'community_number',
                               'COMMUNITY AREA NAME': 'community_area' , 
                               'PERCENT OF HOUSING CROWDED': 'percent_house_crowded', 
                               'PERCENT HOUSEHOLDS BELOW POVERTY': 'percent_house_below_poverty', 
                               'PERCENT AGED 16+ UNEMPLOYED': 'percent_16_unemployed' , 
                               'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA': 'percent_25_without_diploma', 
                               'PERCENT AGED UNDER 18 OR OVER 64': 'percent_dependent', 
                               'PER CAPITA INCOME ' : 'per_capita_income',
                               'HARDSHIP INDEX': 'hardship_index'}, inplace=True)

In [7]:
# Check the columns were renamed.
chicago_census.columns

Index(['community_number', 'community_area', 'percent_house_crowded',
       'percent_house_below_poverty', 'percent_16_unemployed',
       'percent_25_without_diploma', 'percent_dependent', 'per_capita_income',
       'hardship_index'],
      dtype='object')

#### Question 6 
###### Add a new variable to the dataframe called "income".
###### Set `income` to 0 if per_capita_income of the row is less than the mean per_capita_income, otherwise set `income` to 1.
###### Make a new duplicate dataframe called "chicago_knn".

#### Answer:

In [8]:
# Define per_capita_income mean.
mean_per_capita_income = chicago_census['per_capita_income'].mean()
chicago_census['income'] = np.where(chicago_census['per_capita_income'] < mean_per_capita_income,0,1 )
chicago_knn = chicago_census

#### Question 7
###### Look for NAs in `chicago_knn` and impute with either a 0 or the mean.
###### Impute community_member with 0 and hardship_index with mean.
###### Check for NAs after again to make sure the data looks good.

#### Answer:

In [9]:
# Check for NAs.
chicago_knn.isnull().sum(axis=0)

community_number               1
community_area                 0
percent_house_crowded          0
percent_house_below_poverty    0
percent_16_unemployed          0
percent_25_without_diploma     0
percent_dependent              0
per_capita_income              0
hardship_index                 1
income                         0
dtype: int64

In [10]:
# Fill the community number with 0, because the mean will not apply here.
chicago_knn.community_number = chicago_knn.community_number.fillna(0)
# Fill the hardship_index with the mean of the column.
chicago_knn.hardship_index = chicago_knn.hardship_index.fillna(chicago_knn.hardship_index.mean())
# Check for NAs again:
chicago_knn.isnull().sum(axis=0)

community_number               0
community_area                 0
percent_house_crowded          0
percent_house_below_poverty    0
percent_16_unemployed          0
percent_25_without_diploma     0
percent_dependent              0
per_capita_income              0
hardship_index                 0
income                         0
dtype: int64

#### Question 8
##### Check the data types of the columns. 
##### Make sure to only keep numeric columns for kNN. Also, drop `community_number` as it is an ID and we do not want to scale it.
##### Once you ensure that all variables are numeric, transform the target variable `income` to boolean.
##### After you are done, save `chicago_numeric` as `ex_cleaned_knn.csv` for later use.

#### Answer:

In [11]:
# Check for all numeric variables.
chicago_knn.dtypes

community_number               float64
community_area                  object
percent_house_crowded          float64
percent_house_below_poverty    float64
percent_16_unemployed          float64
percent_25_without_diploma     float64
percent_dependent              float64
per_capita_income                int64
hardship_index                 float64
income                           int32
dtype: object

In [12]:
# Drop community area, it is not numeric and community number (this is an ID number).
chicago_numeric = chicago_knn.drop(['community_area','community_number'], axis = 1)

# Check for all numeric variables now.
chicago_knn.dtypes

community_number               float64
community_area                  object
percent_house_crowded          float64
percent_house_below_poverty    float64
percent_16_unemployed          float64
percent_25_without_diploma     float64
percent_dependent              float64
per_capita_income                int64
hardship_index                 float64
income                           int32
dtype: object

In [13]:
chicago_numeric["income"] = np.where(chicago_numeric["income"] == 0, False, True)

In [14]:
chicago_numeric.to_csv("ex_cleaned_knn.csv", index = False)

#### Question 9
###### Once you ensure that all variables are numeric and that the target variable is Boolean, scale the variables using `scale()`.
###### When using `scale()`, remember that you first have to split your predictors from your target.
###### Label your scaled data `ex_X_scaled` (predictors) and `ex_y` (target).
###### Display the first five records in `ex_X_scaled.`

In [15]:
# Split the data into X and y - y is categorical, so it can't scale.
ex_X = chicago_numeric.drop('income', axis = 1)
ex_y = np.array(chicago_numeric['income'])

# Scale X.
ex_X_scaled = scale(ex_X)
print(ex_X_scaled[0:5])

[[ 0.76455091  0.16341619 -0.89126115 -0.18256997 -1.13545603 -0.10981094
  -0.37098756]
 [ 0.79205781 -0.39879857 -0.87784062  0.04020494  0.38438995 -0.16935257
  -0.12381538]
 [-0.3082184   0.19855461 -0.86442008 -0.73093897 -1.86774546  0.6748935
  -1.04188347]
 [-0.41824602 -0.95222873 -0.95836383 -0.59384672 -1.41179167  0.78993668
  -1.1478144 ]
 [-1.27096009 -1.25090533 -1.36097987 -1.35642237 -1.3150742   2.08799731
  -1.53622783]]


## Exercise 2

#### Question 1
###### Split the scaled data into train and test, use a 70 train - 30 test split. 
###### Name your train `ex_X_train` and `ex_y_train` and your test as `ex_X_test` and `ex_y_test`.
###### Don't forget to set the seed as `np.random.seed(2)`.



#### Answer:

In [16]:
# Set the seed.
np.random.seed(2)

# Split into train and test data.
ex_X_train, ex_X_test, ex_y_train, ex_y_test = train_test_split(ex_X_scaled, 
                                                                ex_y, 
                                                                test_size = 0.3) 

#### Question 2
###### Instantiate a kNN model that uses 5 nearest neighbors.
###### Name it `ex_knn`.
###### Fit the classifier to the data.

#### Answer:

In [17]:
# Create KNN classifier.
ex_knn = KNeighborsClassifier(n_neighbors = 5)
# Fit the classifier to the data.
ex_knn.fit(ex_X_train, ex_y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

#### Question 3
###### Predict on the test data. Save as `ex_predictions`.
###### Print out your first 5 predictions, then look at the first 5 actual vs. predicted observations.
###### Name actual vs predicted object as `ex_actual_v_predicted`.

#### Answer:

In [18]:
# Predict on test.
ex_predictions = ex_knn.predict(ex_X_test)
# Print the first 5 predictions.
print(ex_predictions[0:5])

[False False  True False  True]


In [19]:
# Print the first 5 actual vs. predicted.
ex_actual_v_predicted = np.column_stack((ex_y_test, ex_predictions))
print(ex_actual_v_predicted[0:5])

[[False False]
 [False False]
 [ True  True]
 [False False]
 [ True  True]]


#### Question 4
###### Now, let's analyze our model.
###### Build a confusion matrix. Name it ex_cm_knn5.

#### Answer:

In [20]:
# Confusion matrix for knn.
ex_cm_knn5 = confusion_matrix(ex_y_test, ex_predictions)
print(ex_cm_knn5)

[[13  4]
 [ 0  7]]


#### Question 5
###### Now calculate accuracy of our model by comparing our predictions to our actual values. 
###### Round the accuracy to four decimal values.

#### Answer:

In [21]:
print(round(accuracy_score(ex_y_test,ex_predictions),4))

0.8333


#### Question 6
###### Create a dictionary similar to what we created in class, where you will store final accuracy scores of all your classification models.
###### Name it `ex_model_final_dict`.
###### Convert the dictionary to a dataframe and name it as `ex_model_final`.
###### Display the dataframe.
###### Pickle the dataframe `ex_model_final` as `ex_model_final.sav`

In [22]:
# Create a dictionary with accuracy values for our knn model with k = 5.
ex_model_final_dict = {'metrics': ["accuracy"],
                       'values':[round(accuracy_score(ex_y_test, ex_predictions), 4)],
                       'model':['knn_5']}
ex_model_final = pd.DataFrame(data = ex_model_final_dict)

In [23]:
# Pickle `ex_model_final` dataframe.
pickle.dump(ex_model_final, open("ex_model_final.sav","wb" ))